In [12]:
import pandas as pd

In [2]:
import numpy as np
class Machine_Replacement:
    def __init__(self,rep_cost=0.7,nS=6,nA=2):
        self.nS = nS;
        self.nA = nA;
        self.cost = np.linspace(0.1, 0.99,nS);
        self.rep_cost = rep_cost;
    def gen_probability(self):
        self.P = np.zeros((self.nA,self.nS,self.nS));
        for i in range(self.nS):
            for j in range(self.nS):
                if(i<=j):
                    self.P[0,i,j]=(i+1)*(j+1);
                else:
                    continue;
            self.P[0,i,:]=self.P[0,i,:]/np.sum(self.P[0,i,:])
            self.P[1,i,0]=1;
        return self.P;
    def gen_reward(self):
        self.R=np.zeros((self.nA,self.nS,self.nS));
        for i in range(self.nS):
            self.R[0,i,:] = self.cost[i];
            self.R[1,i,0] = self.rep_cost+self.cost[0];
        return self.R;
    def gen_expected_reward(self):
        self.R = np.zeros((self.nA,self.nS));
        for i in range(self.nS):
            self.R[0,i] = self.cost[i];
            self.R[1,i] = self.rep_cost + self.cost[0];
        return self.R;

In [3]:
class beh_pol_sd:
    def __init__(self,P,policy,nS,nA):
        self.P = P
        self.policy = policy
        self.nS = nS;
        self.nA = nA;
    
    def onehot(self):
        pol = np.zeros((self.nS,self.nA));
        for i in range(self.nS):
            pol[i][int(self.policy[i])]=1;
        return pol;
    def find_transition_matrix(self,onehot_encode=1):
        if(onehot_encode==1):
            self.policy = self.onehot()
        T_s_s_next = np.zeros((self.nS,self.nS));
        for s in range(self.nS):
            for s_next in range(self.nS):
                for a in range(self.nA):
                    #print(s,s_next,a);
                    #print(T[a,s,s_next]);
                    T_s_s_next[s,s_next]+=self.P[a,s,s_next]*self.policy[s,a];
        return T_s_s_next;
    def state_distribution_simulated(self,onehot_encode=1):
        P_policy = self.find_transition_matrix(onehot_encode)
        #print(P_policy);
        P_dash = np.append(P_policy - np.eye(self.nS),np.ones((self.nS,1)),axis=1);
        #print(P_dash);
        P_last = np.linalg.pinv(np.transpose(P_dash))[:,-1]
        return P_last;

In [9]:
def one_hot(target_policy,nS,nA):
    tp = np.zeros((nS,nA));
    for j in range(nS):
        tp[j][int(target_policy[j])] = 1;
    return tp;

In [11]:
if __name__=='__main__':
    T,runs = 500000,10
    rep_cost,nS,nA = 0.7,4,2
    mr_obj = Machine_Replacement(rep_cost,nS,nA)
    P,R = mr_obj.gen_probability(),mr_obj.gen_expected_reward();
    alpha = 0.1
    gamma = 0.9
    batch_size = 50;
    q_value_function = np.zeros((int(T/batch_size),runs))
    for run in range(runs):
        Q_table = np.random.random((nS,nA));
        state = 0
        for t in range(1,T+1):
            if(t%batch_size==0):
                policy = np.array([np.argmin(Q_table[s,:]) for s in range(nS)])
                policy = one_hot(policy,nS,nA)
                sd = beh_pol_sd(P,policy,nS,nA).state_distribution_simulated(0);
                qvf = sum([sd[s]*R[np.argmin(Q_table[s,:]),s] for s in range(nS)])
                q_value_function[int(t/batch_size)-1,run] = qvf
            action = np.argmin(Q_table[state,:]);
            next_state = np.argmax(np.random.multinomial(1,P[action,state,:]));
            Q_table[state,action] = Q_table[state,action] + alpha*(R[action,state]+gamma*np.min(Q_table[next_state,:])-Q_table[state,action])
            state = next_state;
        print("one run complete");

one run complete
one run complete
one run complete
one run complete
one run complete
one run complete
one run complete
one run complete
one run complete
one run complete


NameError: name 'pd' is not defined

In [13]:
pd.DataFrame(q_value_function).to_excel('Q_value_function_'+str(nS)+'_state.xlsx');

In [15]:
R

array([[0.1       , 0.39666667, 0.69333333, 0.99      ],
       [0.8       , 0.8       , 0.8       , 0.8       ]])